# 🎨 Low-Level Processing

### Pixels & Filters - The Foundation of Image Analysis

This notebook covers:
- Image filtering (Gaussian, Bilateral)
- Edge detection (Sobel, Canny)
- Histogram operations
- Morphological operations
- Noise reduction

---


In [ ]:
# Install and import libraries
!pip install opencv-python numpy matplotlib scikit-image -q

import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, filters
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries installed and imported successfully!")


## 1. Image Filtering

### Gaussian Blur
Smooths images by averaging nearby pixels with Gaussian weights.

**Formula:** `G(x,y) = (1/(2πσ²)) × exp(-(x²+y²)/(2σ²))`

**Properties:**
- Separable (can be applied as 1D filters)
- Reduces noise
- Blurs edges


In [ ]:
# Load a sample image
image = data.camera()  # Built-in test image

# Apply Gaussian blur with different sigma values
blurred_1 = cv2.GaussianBlur(image, (5, 5), 1.0)
blurred_2 = cv2.GaussianBlur(image, (15, 15), 3.0)
blurred_3 = cv2.GaussianBlur(image, (25, 25), 5.0)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(image, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(blurred_1, cmap='gray')
axes[0, 1].set_title('Gaussian Blur (σ=1.0)')
axes[0, 1].axis('off')

axes[1, 0].imshow(blurred_2, cmap='gray')
axes[1, 0].set_title('Gaussian Blur (σ=3.0)')
axes[1, 0].axis('off')

axes[1, 1].imshow(blurred_3, cmap='gray')
axes[1, 1].set_title('Gaussian Blur (σ=5.0)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Gaussian blur demonstrated!")
print("\nObservation: Larger σ values create more blur")


## 2. Bilateral Filtering

**Edge-preserving smoothing** that uses:
- Spatial proximity (like Gaussian)
- Intensity similarity (range filter)

**Formula:** `w = exp(-spatial²/2σₛ²) × exp(-intensity_diff²/2σᵣ²)`

**Use case:** Smooth noise while preserving edges


In [ ]:
# Compare Gaussian vs Bilateral filtering

# Create image with noise
noisy_image = image.astype(np.float32) + np.random.normal(0, 20, image.shape)
noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

# Apply filters
gaussian_filtered = cv2.GaussianBlur(noisy_image, (15, 15), 3.0)
bilateral_filtered = cv2.bilateralFilter(noisy_image, 15, 80, 80)

# Visualize
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

axes[0].imshow(image, cmap='gray')
axes[0].set_title('Original (Clean)')
axes[0].axis('off')

axes[1].imshow(noisy_image, cmap='gray')
axes[1].set_title('Noisy Image')
axes[1].axis('off')

axes[2].imshow(gaussian_filtered, cmap='gray')
axes[2].set_title('Gaussian Blur\n(Blurs edges)')
axes[2].axis('off')

axes[3].imshow(bilateral_filtered, cmap='gray')
axes[3].set_title('Bilateral Filter\n(Preserves edges)')
axes[3].axis('off')

plt.tight_layout()
plt.show()

print("✅ Bilateral filtering preserves edges while smoothing!")


## 3. Edge Detection

### Sobel Operator
Computes gradient magnitude: `G = √(Gx² + Gy²)`

### Canny Edge Detector
Multi-stage algorithm:
1. Gaussian smoothing
2. Gradient computation
3. Non-maximum suppression
4. Double thresholding + hysteresis


In [ ]:
# Edge Detection Comparison

# Sobel edges
sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
sobel_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

# Canny edges
canny_edges = cv2.Canny(image, 50, 150)

# Laplacian (second derivative)
laplacian = cv2.Laplacian(image, cv2.CV_64F)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(image, cmap='gray')
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(sobel_magnitude, cmap='gray')
axes[0, 1].set_title('Sobel Gradient Magnitude')
axes[0, 1].axis('off')

axes[1, 0].imshow(canny_edges, cmap='gray')
axes[1, 0].set_title('Canny Edges')
axes[1, 0].axis('off')

axes[1, 1].imshow(np.abs(laplacian), cmap='gray')
axes[1, 1].set_title('Laplacian (2nd Derivative)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Edge detection methods compared!")
print("\nKey differences:")
print("- Sobel: Shows gradient magnitude (thick edges)")
print("- Canny: Thin, connected edges (best for most applications)")
print("- Laplacian: Zero-crossings indicate edges")


## 4. Histogram Operations

### Histogram Equalization
Maps intensity values to achieve uniform distribution.

**Formula:** `s = T(r) = (L-1) × Σⱼ₌₀ʳ p(rⱼ)`

Where `p(r)` is the normalized histogram.

### CLAHE (Contrast Limited Adaptive Histogram Equalization)
- Divides image into tiles
- Equalizes each tile separately
- Clips histogram to prevent over-enhancement


In [ ]:
# Histogram Equalization

# Create low-contrast image
low_contrast = cv2.convertScaleAbs(image, alpha=0.5, beta=50)

# Global histogram equalization
equalized = cv2.equalizeHist(low_contrast)

# CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_equalized = clahe.apply(low_contrast)

# Compute histograms
hist_original = cv2.calcHist([low_contrast], [0], None, [256], [0, 256])
hist_equalized = cv2.calcHist([equalized], [0], None, [256], [0, 256])
hist_clahe = cv2.calcHist([clahe_equalized], [0], None, [256], [0, 256])

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(low_contrast, cmap='gray')
axes[0, 0].set_title('Low Contrast Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(equalized, cmap='gray')
axes[0, 1].set_title('Global Histogram Equalization')
axes[0, 1].axis('off')

axes[0, 2].imshow(clahe_equalized, cmap='gray')
axes[0, 2].set_title('CLAHE')
axes[0, 2].axis('off')

axes[1, 0].plot(hist_original)
axes[1, 0].set_title('Original Histogram')
axes[1, 0].set_xlabel('Intensity')
axes[1, 0].set_ylabel('Frequency')

axes[1, 1].plot(hist_equalized)
axes[1, 1].set_title('Equalized Histogram')
axes[1, 1].set_xlabel('Intensity')
axes[1, 1].set_ylabel('Frequency')

axes[1, 2].plot(hist_clahe)
axes[1, 2].set_title('CLAHE Histogram')
axes[1, 2].set_xlabel('Intensity')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("✅ Histogram equalization demonstrated!")
print("\nCLAHE advantages:")
print("- Prevents over-enhancement")
print("- Adapts to local contrast")
print("- Better for images with varying illumination")


## 5. Morphological Operations

**Erosion:** Shrinks foreground objects  
**Dilation:** Expands foreground objects  
**Opening:** Erosion followed by dilation (removes noise)  
**Closing:** Dilation followed by erosion (fills holes)

**Structuring Element:** Defines the shape and size of the operation


In [ ]:
# Morphological Operations

# Create binary image
_, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Define structuring element
kernel = np.ones((5, 5), np.uint8)

# Apply morphological operations
erosion = cv2.erode(binary, kernel, iterations=1)
dilation = cv2.dilate(binary, kernel, iterations=1)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
gradient = cv2.morphologyEx(binary, cv2.MORPH_GRADIENT, kernel)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(binary, cmap='gray')
axes[0, 0].set_title('Original Binary')
axes[0, 0].axis('off')

axes[0, 1].imshow(erosion, cmap='gray')
axes[0, 1].set_title('Erosion')
axes[0, 1].axis('off')

axes[0, 2].imshow(dilation, cmap='gray')
axes[0, 2].set_title('Dilation')
axes[0, 2].axis('off')

axes[1, 0].imshow(opening, cmap='gray')
axes[1, 0].set_title('Opening (Erosion → Dilation)')
axes[1, 0].axis('off')

axes[1, 1].imshow(closing, cmap='gray')
axes[1, 1].set_title('Closing (Dilation → Erosion)')
axes[1, 1].axis('off')

axes[1, 2].imshow(gradient, cmap='gray')
axes[1, 2].set_title('Morphological Gradient')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("✅ Morphological operations demonstrated!")
print("\nApplications:")
print("- Opening: Remove small noise, separate objects")
print("- Closing: Fill holes, connect nearby objects")
print("- Gradient: Edge detection for binary images")


## 📝 Summary

In this notebook, we covered:

1. **Gaussian Blur**: Smoothing with Gaussian weights
2. **Bilateral Filter**: Edge-preserving smoothing
3. **Edge Detection**: Sobel, Canny, Laplacian
4. **Histogram Operations**: Equalization and CLAHE
5. **Morphological Operations**: Erosion, dilation, opening, closing

### Key Takeaways:
- Filtering reduces noise but may blur important features
- Bilateral filter preserves edges while smoothing
- Canny is the standard for edge detection
- CLAHE is better than global equalization for varying illumination
- Morphology is powerful for binary image processing

---

**Next Steps:**
- Experiment with different filter sizes and parameters
- Try edge detection on your own images
- Combine multiple operations for specific tasks
